In [1]:
!pip install gradio # 그라디오 라이브러리를 설치합니다.
!pip install openai # openai 라이브러리를 설치합니다.
!pip install langchain # 랭체인 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.4 MB/s eta 0:

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "자신의 OpenAI KEY를 입력하세요" # 환경변수에 OPENAI_API_KEY를 설정합니다.

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat_llm = ChatOpenAI(model_name="gpt-4")
#chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [4]:
instructions = """
너는 카페에서 주문을 받는 사람처럼 행동해.

[지시]
- 현재 상태에 따라 현재 단계에 맞는 행동을 수행한다.
- 매 응답 시에 [응답 양식]에 맞게 작성한다. 각 항목에 내용이 없을 경우 '해당사항 없음'이라고 표시해
- 마지막 단계까지 수행하였다면 임무를 종료한다.

[단계종류]
단계의 순번이 단계의 순서이다. 각 단계는 ‘정보수집’, ‘호출’, ‘생성’ 중 하나의 행동을 수행해야한다. 각 행동에 대한 가이드는 아래와 같다.
- '정보수집’ : 수집할 정보 항목을 나열하고, 각 항목의 옵션은 괄호() 안에 표기하였다.
- ‘호출' : 호출할 함수명과 인자를 표기하였다.
- ‘생성’ : 생성을 위한 가이드를 작성하였다.
1. 주문
- 정보수집 : 음료종류(아메리카노, 라떼, 녹차), 음료사이즈 (Short, Tall, Grande, Vanti), 컵종류 (매장컵, 개인컵, 일회용컵)
2. 결제
- 정보수집 : 결제방식(현금, 신용카드)
3. 입력
- 호출 : 주문(음료종류, 음료사이즈, 컵종류, 결제방식)
4. 제조
- 생성 : 고객의 요청사항에 맞게 음료를 제조한다. 고객에게 제조한 음료를 소개하는 글을 창의적이고 매력적으로 작성한다.

[행동 종류]
행동은 여러 개의 행동이 동시에 이뤄질 수 있다.
- 단계이동 : 다음단계로 이동한다.
- 정보수집 : 현재단계에서 수집할 정보를 사용자에게 요청한다. 단 수집할 정보가 3개 이상일 경우,  최대 2개로 나누어서 요청한다. 사용자에게 요청할 정보는 [응답 양식]의 'speak' key에  작성한다. 만약 이미 현재단계에서 수집할 정보를 확보하고 있다면, 다음 단계로 이동한다.
- 호출 : 수집한 정보를 호출 양식에 맞게 입력하여 호출한다. 호출할 내용은 [응답 양식]의 'system' key에 파이썬 코드블록으로 작성한다.
- 생성 : 수집한 정보를 바탕으로 가이드에 맞게 작성한다. 생성할 내용은 [응답 양식]의 'system' key에 작성한다. 단 이때 코드블록을 사용하지마.

아래 [응답 양식] 처럼 JSON 반드시 포맷으로 작성해야한다. "수집한정보"인 경우 현재단계뿐만아니라 현재단계까지 수집된 모든 항목과 값까지 모두 기재되어야 한다.
{
    "status" : {
        "현재단계" : "현재단계",
        "수집한정보" : "수집한 정보",
        "수집할정보" : "수집할 정보"
    },
    "thought" : "생각",
    "action" : "행동",
    "speak" : "말하기",
    "system" : "시스템"
}
"""

In [5]:
messages = [
    SystemMessage(content=instructions),
    HumanMessage(content="안녕")
]

ai_message = chat_llm(messages)

In [6]:
ai_message

AIMessage(content='{\n    "status" : {\n        "현재단계" : "주문",\n        "수집한정보" : "해당사항 없음",\n        "수집할정보" : "음료종류, 음료사이즈"\n    },\n    "thought" : "첫 번째 단계인 주문 단계에 있으므로, 고객에게 음료 종류와 사이즈를 물어보겠습니다.",\n    "action" : "정보수집",\n    "speak" : "안녕하세요, 어떤 음료를 주문하시겠습니까? (아메리카노, 라떼, 녹차) 그리고 사이즈는 어떻게 드릴까요? (Short, Tall, Grande, Vanti)",\n    "system" : "해당사항 없음"\n}', additional_kwargs={}, example=False)

In [7]:
import gradio as gr  # 그라디오 라이브러리를 불러옵니다.
import json

In [8]:
print(instructions)


너는 카페에서 주문을 받는 사람처럼 행동해.

[지시]
- 현재 상태에 따라 현재 단계에 맞는 행동을 수행한다.
- 매 응답 시에 [응답 양식]에 맞게 작성한다. 각 항목에 내용이 없을 경우 '해당사항 없음'이라고 표시해
- 마지막 단계까지 수행하였다면 임무를 종료한다.

[단계종류]
단계의 순번이 단계의 순서이다. 각 단계는 ‘정보수집’, ‘호출’, ‘생성’ 중 하나의 행동을 수행해야한다. 각 행동에 대한 가이드는 아래와 같다.
- '정보수집’ : 수집할 정보 항목을 나열하고, 각 항목의 옵션은 괄호() 안에 표기하였다.
- ‘호출' : 호출할 함수명과 인자를 표기하였다.
- ‘생성’ : 생성을 위한 가이드를 작성하였다.
1. 주문
- 정보수집 : 음료종류(아메리카노, 라떼, 녹차), 음료사이즈 (Short, Tall, Grande, Vanti), 컵종류 (매장컵, 개인컵, 일회용컵)
2. 결제
- 정보수집 : 결제방식(현금, 신용카드)
3. 입력
- 호출 : 주문(음료종류, 음료사이즈, 컵종류, 결제방식)
4. 제조
- 생성 : 고객의 요청사항에 맞게 음료를 제조한다. 고객에게 제조한 음료를 소개하는 글을 창의적이고 매력적으로 작성한다.

[행동 종류]
행동은 여러 개의 행동이 동시에 이뤄질 수 있다.
- 단계이동 : 다음단계로 이동한다.
- 정보수집 : 현재단계에서 수집할 정보를 사용자에게 요청한다. 단 수집할 정보가 3개 이상일 경우,  최대 2개로 나누어서 요청한다. 사용자에게 요청할 정보는 [응답 양식]의 'speak' key에  작성한다. 만약 이미 현재단계에서 수집할 정보를 확보하고 있다면, 다음 단계로 이동한다.
- 호출 : 수집한 정보를 호출 양식에 맞게 입력하여 호출한다. 호출할 내용은 [응답 양식]의 'system' key에 파이썬 코드블록으로 작성한다.
- 생성 : 수집한 정보를 바탕으로 가이드에 맞게 작성한다. 생성할 내용은 [응답 양식]의 'system' key에 작성한다. 단 이때 코드블록을 사용하지마.

아래 [응답 양식] 처럼

In [9]:
messages = []

messages.append(SystemMessage(content=instructions))

def respond(user_message, chat_history):  # 채팅봇의 응답을 처리하는 함수를 정의합니다.

    global messages

    messages.append(HumanMessage(content=user_message))

    print("### input ###")
    print(messages)

    result = chat_llm(messages)

    print("### result ###")
    print(result)

    status_text = result.content

    status_object = json.loads(status_text)

    ai_message = status_object['speak']

    messages.append(AIMessage(content=status_text))

    chat_history.append((user_message, ai_message))  # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가합니다.

    return "", chat_history, status_text  # 수정된 채팅 기록을 반환합니다.

with gr.Blocks() as demo:  # gr.Blocks()를 사용하여 인터페이스를 생성합니다.
    status_text = gr.Textbox(label="상태")
    chatbot = gr.Chatbot(label="채팅창")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    msg = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    msg.submit(respond, [msg, chatbot], [msg, chatbot, status_text])  # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    clear.click(lambda: None, None, chatbot, queue=False)  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.

demo.launch(debug=True)  # 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

### input ###
[SystemMessage(content='\n너는 카페에서 주문을 받는 사람처럼 행동해.\n\n[지시]\n- 현재 상태에 따라 현재 단계에 맞는 행동을 수행한다.\n- 매 응답 시에 [응답 양식]에 맞게 작성한다. 각 항목에 내용이 없을 경우 \'해당사항 없음\'이라고 표시해\n- 마지막 단계까지 수행하였다면 임무를 종료한다.\n\n[단계종류]\n단계의 순번이 단계의 순서이다. 각 단계는 ‘정보수집’, ‘호출’, ‘생성’ 중 하나의 행동을 수행해야한다. 각 행동에 대한 가이드는 아래와 같다.\n- \'정보수집’ : 수집할 정보 항목을 나열하고, 각 항목의 옵션은 괄호() 안에 표기하였다.\n- ‘호출\' : 호출할 함수명과 인자를 표기하였다.\n- ‘생성’ : 생성을 위한 가이드를 작성하였다.\n1. 주문\n- 정보수집 : 음료종류(아메리카노, 라떼, 녹차), 음료사이즈 (Short, Tall, Grande, Vanti), 컵종류 (매장컵, 개인컵, 일회용컵)\n2. 결제\n- 정보수집 : 결제방식(현금, 신용카드)\n3. 입력\n- 호출 : 주문(음료종류, 음료사이즈, 컵종류, 결제방식)\n4. 제조\n- 생성 : 고객의 요청사항에 맞게 음료를 제조한다. 고객에게 제조한 음료를 소개하는 글을 창의적이고 매력적으로 작성한다.\n\n[행동 종류]\n행동은 여러 개의 행동이 동시에 이뤄질 수 있다.\n- 단계이동 : 다음단계로 이동한다.\n- 정보수집 : 현재단계에서 수집할 정보를 사용자에게 요청한다. 단 수집할 정보가 3개 이상일 경우,  최대 2개로 나누어서 요청한다. 사용자에게 요청할 정보는 [응답 양식]의 \'speak\' key에  작성한다. 만약 이미 현재단계에서 수집할 정보를 확보하고 있다면, 다음 단계로 이동한다.\n- 호출 : 수집한 정보를 호출 양식에 맞게 입력하여 호출한다. 호출할 내용은 [응답 양식]의 \'system\' key에 파이썬 코드블록으로 작성한다.\n- 생성 : 수집한 정보를 바탕으로 가이드에 맞게 작성

In [ ]:
role_prompt = "너는 카페에서 주문을 받는 사람처럼 행동해."

instruction_prompt = """

[지시]
- 현재 상태에 따라 현재 단계에 맞는 행동을 수행한다.
- 매 응답 시에 [응답 양식]에 맞게 작성한다. 각 항목에 내용이 없을 경우 '해당사항 없음'이라고 표시해
- 마지막 단계까지 수행하였다면 임무를 종료한다.
"""

step_prompt = """

[단계종류]
단계의 순번이 단계의 순서이다. 각 단계는 ‘정보수집’, ‘호출’, ‘생성’ 중 하나의 행동을 수행해야한다. 각 행동에 대한 가이드는 아래와 같다.
- '정보수집’ : 수집할 정보 항목을 나열하고, 각 항목의 옵션은 괄호() 안에 표기하였다.
- ‘호출' : 호출할 함수명과 인자를 표기하였다.
- ‘생성’ : 생성을 위한 가이드를 작성하였다.
1. 주문
- 정보수집 : 음료종류(아메리카노, 라떼, 녹차), 음료사이즈 (Short, Tall, Grande, Vanti), 컵종류 (매장컵, 개인컵, 일회용컵)
2. 결제
- 정보수집 : 결제방식(현금, 신용카드)
3. 입력
- 호출 : 주문(음료종류, 음료사이즈, 컵종류, 결제방식)
4. 제조
- 생성 : 고객의 요청사항에 맞게 음료를 제조한다. 고객에게 제조한 음료를 소개하는 글을 창의적이고 매력적으로 작성한다.
"""

action_response_prompt = """

[행동 종류]
행동은 여러 개의 행동이 동시에 이뤄질 수 있다.
- 단계이동 : 다음단계로 이동한다.
- 정보수집 : 현재단계에서 수집할 정보를 사용자에게 요청한다. 단 수집할 정보가 3개 이상일 경우,  최대 2개로 나누어서 요청한다. 사용자에게 요청할 정보는 [응답 양식]의 'speak' key에  작성한다. 만약 이미 현재단계에서 수집할 정보를 확보하고 있다면, 다음 단계로 이동한다.
- 호출 : 수집한 정보를 호출 양식에 맞게 입력하여 호출한다. 호출할 내용은 [응답 양식]의 'system' key에 파이썬 코드블록으로 작성한다.
- 생성 : 수집한 정보를 바탕으로 가이드에 맞게 작성한다. 생성할 내용은 [응답 양식]의 'system' key에 작성한다. 단 이때 코드블록을 사용하지마.

아래 [응답 양식] 처럼 JSON 반드시 포맷으로 작성해야한다. "수집한정보"인 경우 현재단계뿐만아니라 현재단계까지 수집된 모든 항목과 값까지 모두 기재되어야 한다.
{
    "status" : {
        "현재단계" : "현재단계",
        "수집한정보" : "수집한 정보",
        "수집할정보" : "수집할 정보"
    },
    "thought" : "생각",
    "action" : "행동",
    "speak" : "말하기",
    "system" : "시스템"
}
"""

In [ ]:
messages = []

def respond(role_val, step_val, user_message, chat_history):  # 채팅봇의 응답을 처리하는 함수를 정의합니다.

    global messages

    if len(messages) == 0:
        system_message = role_val + instruction_prompt + step_val + action_response_prompt

        messages.append(SystemMessage(content=system_message))

    messages.append(HumanMessage(content=user_message))

    result = chat_llm(messages)

    status_text = result.content

    status_object = json.loads(status_text)

    ai_message = status_object['speak']

    messages.append(AIMessage(content=status_text))

    chat_history.append((user_message, ai_message))  # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가합니다.

    return "", chat_history, status_text  # 수정된 채팅 기록을 반환합니다.


def reset_prompt_status():
    messages = []

with gr.Blocks() as demo:  # gr.Blocks()를 사용하여 인터페이스를 생성합니다.

    role = gr.Textbox(label="역할 정의", value = role_prompt)
    step = gr.Textbox(label="단계 정의", value = step_prompt)

    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot(label="채팅창")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
            msg = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.

        with gr.Column():
            status_text = gr.Textbox(label="상태")
            clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    msg.submit(respond, [role, step, msg, chatbot], [msg, chatbot, status_text])  # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    clear.click(reset_prompt_status, None, chatbot, queue=False)  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.

demo.launch(debug=True)  # 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.

역할 : 너는 업무지원받는 사람처럼 행동해.

[단계종류]
단계의 순번이 단계의 순서이다. 각 단계는 ‘정보수집’, ‘호출’, ‘생성’ 중 하나의 행동을 수행해야한다. 각 행동에 대한 가이드는 아래와 같다.
- '정보수집’ : 수집할 정보 항목을 나열하고, 각 항목의 옵션은 괄호() 안에 표기하였다.
- ‘호출' : 호출할 함수명과 인자를 표기하였다.
- ‘생성’ : 생성을 위한 가이드를 작성하였다.

1. 요청
- 정보수집 : 휴가일, 휴가종류(연차, 방학, 병가), 휴가타입(종일, 오전반차, 오후반차)
2. 결재라인
- 정보수집 : 검토(팀장, 부서장)
3. 입력
- 호출 : 휴가신청(휴가일, 휴가종류, 휴가타입)
